In [2]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# other library
import numpy as np
import pandas as pd
import random
import os
from skimage import io
import copy

import importlib
import utils
importlib.reload(utils)
from utils import (
    ImgDataset, VAE, AE, 
    change_to_submit_format_df, output_to_csv, training
)

# Inference

In [3]:
SEED = 5566
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [4]:
MODEL_PATH = "./AE.pth"
DATA_PATH = './trainX.npy'
img_channels = 3
latent_dim = 128
feature_dim = 2*2*256
best_AE = AE(
    img_channels, feature_dim, 
    latent_dim)
best_AE.load_state_dict(
    torch.load(MODEL_PATH)
)

<All keys matched successfully>

In [5]:
BATCH_SIZE = 128
transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor()
])

data = torch.from_numpy(
    np.load(DATA_PATH)
).float()

train_dataset = ImgDataset(data, transform)

latent_space = []
test_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
best_AE.to(device)
best_AE.eval()
for idx, img in enumerate(test_loader):
    img = img.to(device, dtype=torch.float)
    img_out, img_latent = best_AE(img)
    img_latent = img_latent.detach().cpu().numpy()
    for i in range(img_latent.shape[0]):
        latent_space.append(img_latent[i])
latent_space = np.asarray(latent_space)

In [6]:
SUBMIT_PATH = "./prediction.csv"

#PCA -> TSNE -> KMeans
pca = PCA(n_components=45, copy=True, whiten=True, svd_solver='full', random_state = SEED)
latent_pca = pca.fit_transform(latent_space)

tsne = TSNE(random_state = SEED)
tsne_x = tsne.fit_transform(latent_pca)

result = KMeans(n_clusters=2, random_state = SEED, max_iter=1000).fit(tsne_x)
submit_df = change_to_submit_format_df(1 - result.labels_)

output_to_csv(submit_df, SUBMIT_PATH)

# Training record

In [ ]:
BATCH_SIZE = 128
transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.ToTensor()
])

data = torch.from_numpy(
    np.load(DATA_PATH)
    ).float()
train_dataset = ImgDataset(data, transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
img_channels = 3
latent_dim = 128
feature_dim = 2*2*256 #output dimension of encoder stage, 2 x 2 x 256
AE_model = AE(
    img_channels, feature_dim, 
    latent_dim)
AE_model = AE_model.to(device)
NUM_EPOCH = 300
optimizer = torch.optim.Adam(AE_model.parameters(), lr=0.001, weight_decay=1e-5)

best_AE = training(train_loader, AE_model, device, optimizer, NUM_EPOCH)

=== start training, parameter total:1644512, trainable:1644512
Epoch 1  | Loss:429.3778
Epoch 2  | Loss:305.7886
Epoch 3  | Loss:278.2666
Epoch 4  | Loss:259.1717
Epoch 5  | Loss:250.0434
Epoch 6  | Loss:240.3452
Epoch 7  | Loss:232.0494
Epoch 8  | Loss:226.3041
Epoch 9  | Loss:219.6001
Epoch 10  | Loss:213.3392
Epoch 11  | Loss:209.9081
Epoch 12  | Loss:206.4133
Epoch 13  | Loss:205.2126
Epoch 14  | Loss:200.5008
Epoch 15  | Loss:197.0853
Epoch 16  | Loss:195.8938
Epoch 17  | Loss:195.1764
Epoch 18  | Loss:191.9939
Epoch 19  | Loss:191.1830
Epoch 20  | Loss:188.4077
Epoch 21  | Loss:186.5157
Epoch 22  | Loss:187.3844
Epoch 23  | Loss:184.3080
Epoch 24  | Loss:182.4283
Epoch 25  | Loss:181.6893
Epoch 26  | Loss:182.2262
Epoch 27  | Loss:179.4738
Epoch 28  | Loss:178.7760
Epoch 29  | Loss:177.7811
Epoch 30  | Loss:176.3618
Epoch 31  | Loss:174.0466
Epoch 32  | Loss:174.8303
Epoch 33  | Loss:173.7862
Epoch 34  | Loss:172.5193
Epoch 35  | Loss:171.5092
Epoch 36  | Loss:171.5692
Epoch 37  